In [13]:
import requests
import pandas as pd
import folium 
from folium.plugins import MarkerCluster, MiniMap

In [16]:
API_KEY = '8b50343fe07230c95dea834f1b4e9559'

In [149]:
def add2latlng(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    header = {'Authorization': 'KakaoAK ' + API_KEY}
 
    r = requests.get(url, headers=header)
 
    if r.status_code == 200:
        documents = r.json()["documents"]
        if len(documents) > 0:
            result_address = r.json()["documents"][0]["address"]
            if result_address:
                return result_address["y"], result_address["x"]
            else:
                print ("No data:", address)
                return None
        else:
            print ("No data:", address)
            return None
    else:
        print ("ERROR[" + str(r.status_code) + "]")
        return None


# 주소 데이터 로드

In [219]:
def get_lat_lng(df):
    df['latitude'] = 0.0
    df['longitude'] = 0.0
    
    for i, row in enumerate(df.iterrows()):
        address = row[1]['주소']
        
        lat_lng = add2latlng(address)
    
        if lat_lng:
            df.at[i, 'latitude'] = lat_lng[0]
            df.at[i, 'longitude'] = lat_lng[1]
        else:
            print("Error:", i, address)

## 베스킨라빈스

In [220]:
df_baskin = pd.read_csv('data/베스킨라빈스.csv')

In [221]:
get_lat_lng(df_baskin)

No data: 서울 서초구 강남대로 지하 390
Error: 0 서울 서초구 강남대로 지하 390
No data: 서울 서초구 방배3동 1001-31
Error: 18 서울 서초구 방배3동 1001-31
No data: 서울 서초구 태봉로 70, 107-1호
Error: 28 서울 서초구 태봉로 70, 107-1호
No data: 서울 강남구 자곡동 383
Error: 72 서울 강남구 자곡동 383
No data: 서울 용산구 효창동 13-2
Error: 74 서울 용산구 효창동 13-2
No data: 서울 성동구 홍익동 550 센트라스 상가 L동 B19호
Error: 77 서울 성동구 홍익동 550 센트라스 상가 L동 B19호
No data: 서울 성북구 인촌로24길 25 (안암동5가)
Error: 126 서울 성북구 인촌로24길 25 (안암동5가)
No data: 서울 영등포구 당산로 214 (당산동5가, 당산 삼성 래미안)111-2, 111-3호
Error: 171 서울 영등포구 당산로 214 (당산동5가, 당산 삼성 래미안)111-2, 111-3호
No data: 서울 구로구 신도림동 436-7 큐븨스테이트 오피스텔 상가 1층
Error: 200 서울 구로구 신도림동 436-7 큐븨스테이트 오피스텔 상가 1층
No data: 서울 강북구 도봉로 8길 5
Error: 204 서울 강북구 도봉로 8길 5
No data: 서울 강동구 고덕동 212 고덕그라시움아파트 상가4동 1층
Error: 211 서울 강동구 고덕동 212 고덕그라시움아파트 상가4동 1층
No data: 서울 강동구 아리수로 427 106호
Error: 238 서울 강동구 아리수로 427 106호


In [251]:
df_baskin = df_baskin[df_baskin['latitude'] > 0.0]

In [252]:
len(df_baskin)

285

## 메가커피

In [222]:
df_mega = pd.read_csv('data/메가커피.csv')

In [248]:
get_lat_lng(df_mega)

In [246]:
add2latlng('서울특별시 서대문구 연희로 98')

('37.5666191929416', '126.931000128114')

In [247]:
df_mega.at[136, '주소'] = '서울특별시 서대문구 연희로 98'

In [233]:
df_mega.iloc[35]

주소           서울특별시 송파구 법원로4길 6
latitude                     0
longitude                    0
Name: 35, dtype: object

In [122]:
df_mega = df_mega[df_mega['latitude'] > 0.0]

In [249]:
len(df_mega)

257

In [250]:
df_mega.to_csv('메가커피2.csv', index=False)

## 파리바게트

In [142]:
df_paris = pd.read_csv('data/파리바게트.csv')

In [151]:
get_lat_lng(df_paris)

0 서울특별시 강서구 양천로 657
1 서울특별시 강서구 염창동 250-17
2 서울 양천구 목동중앙북로 31
3 서울 양천구 목동중앙북로 73 102호
4 서울특별시 강서구 염창동 282-22
5 서울특별시 등촌2동 512-7
6 서울 강서구 공항대로 425
7 서울특별시 강서구 등촌동 628
8 서울특별시 강서구 화곡동 1117-19
9 서울 양천구 목동중앙본로7길 45 목동보미리즌빌 1층 103호
10 서울특별시 강서구 가양2동 1479-4
11 서울 양천구 목동서로2길 22 1층
12 서울 양천구 목동중앙로 95 1층
13 서울특별시 강서구 등촌동 697-1
14 서울 양천구 목동서로 67
15 서울 양천구 목동중앙로 39 1층
16 서울 양천구 목동동로 427 1층
17 서울특별시 강서구 화곡본동 110-66
18 서울특별시 강서구 화곡동 836-1
19 서울특별시 강서구 등촌동 684-3
20 서울 양천구 목동서로 155 1층
21 서울특별시 강서구 내발산동 678-9
22 서울 영등포구 양평로 122 1층
23 서울특별시 강서구 화곡동 873-1
24 서울 양천구 목동로 210 1층
25 서울 강서구 마곡중앙6로 69 1층
26 서울특별시 강서구 화곡동 343-2
27 서울 마포구 월드컵로 204
28 서울 양천구 오목로 218 1층
29 서울 양천구 오목로 320 1층
30 서울 마포구 월드컵북로 396 ㈜문화방송
31 서울 마포구 월드컵북로 396
32 서울특별시 강서구 화곡동 1068-32
33 서울 영등포구 양평로 67 포스빌 상가 1층
34 서울특별시 마포구 월드컵북로 497 1층 105호~107호(상암동)
35 서울특별시 양천구 오목로 149
36 서울특별시 강서구 화곡1동 361-1
37 서울 마포구 월드컵북로 218
38 서울 양천구 목동동로12길 54
39 서울특별시 강서구 내발산동 750-11
40 서울 양천구 목동서로 295 1층
41 서울특별시 영등포구 영등포로 33
42 서울 영등포구 국회대로 553
43 서울특별시 강서구 

In [152]:
df_paris[df_paris['latitude'] == 0.0]

,주소,latitude,longitude
71,서울특별시 영등포구 영등포동3가 9-3,0.0,0.0
140,"서울특별시 은평구 은평로 237 1, 2층",0.0,0.0
165,서울특별시 금천구 가산디지털1로 145 145,0.0,0.0
173,"서울특별시 동작구 상도동 20-8 은혜발당 103,104,105",0.0,0.0
189,서울 동작구 흑석로 84 102관 B117호,0.0,0.0
224,서울특별시 관악구 청림6길,0.0,0.0
265,서울특별시 관악구 남현동 1061-25 26 지상 1층 가호,0.0,0.0
287,"서울 강남구 압구정로18길 30 1,2층",0.0,0.0
326,"서울특별시 성동구 성수동1가 715 상가동 1층 105, 106, 107호",0.0,0.0
359,서울특별시 서초구 남부순환로356길 2 1층,0.0,0.0


In [153]:
df_paris = df_paris[df_paris['latitude'] > 0.0]

In [253]:
len(df_paris)

508

# 지도 생성

In [254]:
def add_markers(df, color):
    for i, row in enumerate(df.iterrows()):
        address = row[1]['주소']
        latitude = row[1]['latitude']
        longitude = row[1]['longitude']

        if latitude > 0.0 and longitude > 0.0:
            #marker_cluster.add_child(folium.Marker(location=[latitude, longitude], popup='베스킨라빈스')) 
            folium.Marker(location=[latitude, longitude], popup=address, icon=folium.Icon(color=color)).add_to(map_hs)

In [255]:
map_hs = folium.Map((37.5498849819149, 126.874971203942), zoom_start = 13) 
# marker_cluster = MarkerCluster()

add_markers(df_baskin, 'pink')
add_markers(df_mega, 'gray')
add_markers(df_paris, 'blue')

In [256]:
from geopy.distance import geodesic

In [257]:
df_baskin['closest_mega'] = 100000

In [258]:
df_paris['closest_mega'] = 100000

In [259]:
def get_min_distance(latitude, longitude):
    min_dist = 1000000
    
    for row in df_mega.iterrows():
        dist = geodesic((latitude, longitude), (row[1]['latitude'], row[1]['longitude'])).meters
        if dist < min_dist:
            min_dist = dist
    
    return min_dist

In [260]:
for i, row in df_baskin.iterrows():
    latitude = row[1]
    longitude = row[2]
    
    if latitude > 0.0 and longitude > 0.0:
        min_dist = get_min_distance(latitude, longitude)
        
        df_baskin.at[i, 'closest_mega'] = min_dist

In [261]:
for i, row in df_paris.iterrows():
    latitude = row[1]
    longitude = row[2]
    
    if latitude > 0.0 and longitude > 0.0:
        min_dist = get_min_distance(latitude, longitude)
        
        df_paris.at[i, 'closest_mega'] = min_dist

In [262]:
MIN_DISTANCE = 500

In [263]:
df_baskin_no_mega = df_baskin[df_baskin['closest_mega'] > MIN_DISTANCE]

In [264]:
df_paris_no_mega = df_paris[df_paris['closest_mega'] > MIN_DISTANCE]

In [265]:
len(df_baskin_no_mega)

94

In [266]:
len(df_paris_no_mega)

227

In [267]:
for row in df_baskin_no_mega.iterrows():
    folium.Circle([row[1]['latitude'], row[1]['longitude']], 
                  radius=500,
                  fill=True,
                  fill_opacity=0.2,
                 ).add_to(map_hs)

In [268]:
for row in df_paris_no_mega.iterrows():
    folium.Circle([row[1]['latitude'], row[1]['longitude']], 
                  radius=500,
                  fill=True,
                  fill_opacity=0.2,
                 ).add_to(map_hs)

# 배포

In [272]:
FILENAME = 'v1.0.2.html'
map_hs.save(FILENAME)

In [273]:
import paramiko 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('playground21.cafe24.com', username="playground21", password="playground21wp")
sftp = ssh.open_sftp()
sftp.put(FILENAME, 'www/wp-content/franchise/' + FILENAME)
sftp.close()
ssh.close()

print('http://playground21.cafe24.com/wp-content/franchise/' + FILENAME)

http://playground21.cafe24.com/wp-content/franchise/v1.0.2.html
